In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from math import ceil
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# Model configuration
img_width, img_height = 28, 28
batch_size = 250
no_epochs = 20
no_classes = 10
validation_split = 0.2
verbosity = 1
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr



# Load MNIST dataset
(input_train, target_train), (input_test, target_test) = mnist.load_data()

# Reshape data
input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 1)
input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 1)
input_shape = (img_width, img_height, 1)
    
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

Using TensorFlow backend.


## Width and Height Shift

In [2]:
# specify the width and height shift arguments
width_shift_val = 0.5
height_shift_val = 0.5

# create the class object
datagen = ImageDataGenerator(width_shift_range=width_shift_val, height_shift_range=height_shift_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 9s 39ms/step - loss: 1.6003 - acc: 0.4262
Epoch 2/20
240/240 [==============================] - 7s 31ms/step - loss: 0.9962 - acc: 0.6531
Epoch 3/20
240/240 [==============================] - 7s 31ms/step - loss: 0.8305 - acc: 0.7115
Epoch 4/20
240/240 [==============================] - 8s 31ms/step - loss: 0.7461 - acc: 0.7423
Epoch 5/20
240/240 [==============================] - 8s 33ms/step - loss: 0.6944 - acc: 0.7610
Epoch 6/20
240/240 [==============================] - 8s 35ms/step - loss: 0.6514 - acc: 0.7746
Epoch 7/20
240/240 [==============================] - 8s 31ms/step - loss: 0.6210 - acc: 0.7846
Epoch 8/20
240/240 [==============================] - 8s 33ms/step - loss: 0.5944 - acc: 0.7936
Epoch 9/20
240/240 [==============================] - 8s 34ms/step - loss: 0.5753 - acc: 0.7998
Epoch 10/20
240/240 [==============================] - 8s 32ms/step - loss: 0.5623 - acc: 0.8050
Epoch 11/20
240/240 [==================

In [4]:
# specify the width an height shift arguments
width_shift_val = 0.5
height_shift_val = 0.5

# create the class object
datagenn = ImageDataGenerator(width_shift_range=width_shift_val, height_shift_range=height_shift_val)

# fit the generator
datagenn.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagenn.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.37746840505599977 / Test accuracy: 0.8685


## Rotation

In [2]:
# specify the maximum rotation_range angle
rotation_range_val = 90

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 9s 37ms/step - loss: 0.6892 - acc: 0.7789
Epoch 2/20
240/240 [==============================] - 6s 27ms/step - loss: 0.2573 - acc: 0.9196
Epoch 3/20
240/240 [==============================] - 7s 29ms/step - loss: 0.1819 - acc: 0.9437
Epoch 4/20
240/240 [==============================] - 7s 29ms/step - loss: 0.1464 - acc: 0.9533
Epoch 5/20
240/240 [==============================] - 7s 29ms/step - loss: 0.1261 - acc: 0.9608
Epoch 6/20
240/240 [==============================] - 7s 29ms/step - loss: 0.1159 - acc: 0.9628
Epoch 7/20
240/240 [==============================] - 7s 29ms/step - loss: 0.1028 - acc: 0.9675
Epoch 8/20
240/240 [==============================] - 7s 29ms/step - loss: 0.0958 - acc: 0.9705
Epoch 9/20
240/240 [==============================] - 7s 29ms/step - loss: 0.0905 - acc: 0.9717
Epoch 10/20
240/240 [==============================] - 7s 29ms/step - loss: 0.0850 - acc: 0.9732
Epoch 11/20
240/240 [==================

In [6]:
# specify the maximum rotation_range angle
rotation_range_val = 90

# create the class object
datagenn = ImageDataGenerator(rotation_range=rotation_range_val)

# fit the generator
datagenn.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagenn.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.04677017239322886 / Test accuracy: 0.9845


## Shear

In [2]:
# specify the shear argument
shear_range_val=85

# create the class object
datagen = ImageDataGenerator(shear_range=shear_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 9s 36ms/step - loss: 0.6579 - acc: 0.7864
Epoch 2/20
240/240 [==============================] - 6s 26ms/step - loss: 0.2950 - acc: 0.9053
Epoch 3/20
240/240 [==============================] - 6s 26ms/step - loss: 0.2435 - acc: 0.9223
Epoch 4/20
240/240 [==============================] - 6s 26ms/step - loss: 0.2116 - acc: 0.9326
Epoch 5/20
240/240 [==============================] - 7s 28ms/step - loss: 0.1876 - acc: 0.9403
Epoch 6/20
240/240 [==============================] - 7s 28ms/step - loss: 0.1694 - acc: 0.9457
Epoch 7/20
240/240 [==============================] - 7s 27ms/step - loss: 0.1560 - acc: 0.9503
Epoch 8/20
240/240 [==============================] - 7s 29ms/step - loss: 0.1478 - acc: 0.9529
Epoch 9/20
240/240 [==============================] - 7s 27ms/step - loss: 0.1415 - acc: 0.9543
Epoch 10/20
240/240 [==============================] - 6s 27ms/step - loss: 0.1322 - acc: 0.9571
Epoch 11/20
240/240 [==================

In [3]:
# specify the shear argument
shear_range_val=85

# create the class object
datagenn = ImageDataGenerator(shear_range=shear_range_val)

# fit the generator
datagenn.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagenn.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.08330592928454279 / Test accuracy: 0.9727


## Zoom

In [2]:
# specify the zoom argument
zoom_range_val=[2.5,3.5]

# create the class object
datagen = ImageDataGenerator(zoom_range=zoom_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 9s 38ms/step - loss: 0.7416 - acc: 0.7620
Epoch 2/20
240/240 [==============================] - 6s 27ms/step - loss: 0.3320 - acc: 0.8947
Epoch 3/20
240/240 [==============================] - 7s 29ms/step - loss: 0.2771 - acc: 0.9122
Epoch 4/20
240/240 [==============================] - 7s 28ms/step - loss: 0.2476 - acc: 0.9197
Epoch 5/20
240/240 [==============================] - 7s 29ms/step - loss: 0.2253 - acc: 0.9281
Epoch 6/20
240/240 [==============================] - 7s 29ms/step - loss: 0.2123 - acc: 0.9330
Epoch 7/20
240/240 [==============================] - 7s 28ms/step - loss: 0.1998 - acc: 0.9361
Epoch 8/20
240/240 [==============================] - 7s 28ms/step - loss: 0.1934 - acc: 0.9377
Epoch 9/20
240/240 [==============================] - 7s 28ms/step - loss: 0.1821 - acc: 0.9402
Epoch 10/20
240/240 [==============================] - 7s 28ms/step - loss: 0.1766 - acc: 0.9421
Epoch 11/20
240/240 [==================

In [3]:
# specify the zoom argument
zoom_range_val=[2.5,3.5]

# create the class object
datagenn = ImageDataGenerator(zoom_range=zoom_range_val)

# fit the generator
datagenn.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagenn.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.0901137893974781 / Test accuracy: 0.9718


## Rotate + W&H Shift

In [2]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 10s 42ms/step - loss: 1.2404 - acc: 0.5764
Epoch 2/20
240/240 [==============================] - 8s 34ms/step - loss: 0.5543 - acc: 0.8242
Epoch 3/20
240/240 [==============================] - 8s 34ms/step - loss: 0.4030 - acc: 0.8736
Epoch 4/20
240/240 [==============================] - 8s 33ms/step - loss: 0.3298 - acc: 0.8976
Epoch 5/20
240/240 [==============================] - 8s 35ms/step - loss: 0.2905 - acc: 0.9080
Epoch 6/20
240/240 [==============================] - 8s 35ms/step - loss: 0.2602 - acc: 0.9182
Epoch 7/20
240/240 [==============================] - 8s 34ms/step - loss: 0.2365 - acc: 0.9249
Epoch 8/20
240/240 [==============================] - 8s 35ms/step - loss: 0.2240 - acc: 0.9300
Epoch 9/20
240/240 [==============================] - 8s 35ms/step - loss: 0.2078 - acc: 0.9350
Epoch 10/20
240/240 [==============================] - 8s 34ms/step - loss: 0.1984 - acc: 0.9384
Epoch 11/20
240/240 [=================

In [3]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# create the class object
datagenn = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val)

# fit the generator
datagenn.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagenn.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.10059675966203213 / Test accuracy: 0.9679


## Rotate + W&H Shift + Shear

In [2]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# specify the shear argument
shear_range_val = 45

# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val, shear_range=shear_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 11s 44ms/step - loss: 1.5015 - acc: 0.4783
Epoch 2/20
240/240 [==============================] - 8s 35ms/step - loss: 0.8268 - acc: 0.7343
Epoch 3/20
240/240 [==============================] - 8s 35ms/step - loss: 0.6240 - acc: 0.8020
Epoch 4/20
240/240 [==============================] - 8s 35ms/step - loss: 0.5346 - acc: 0.8327
Epoch 5/20
240/240 [==============================] - 9s 38ms/step - loss: 0.4762 - acc: 0.8512
Epoch 6/20
240/240 [==============================] - 9s 36ms/step - loss: 0.4313 - acc: 0.8632
Epoch 7/20
240/240 [==============================] - 8s 35ms/step - loss: 0.3957 - acc: 0.8757
Epoch 8/20
240/240 [==============================] - 10s 41ms/step - loss: 0.3693 - acc: 0.8835
Epoch 9/20
240/240 [==============================] - 9s 37ms/step - loss: 0.3588 - acc: 0.8871
Epoch 10/20
240/240 [==============================] - 9s 36ms/step - loss: 0.3387 - acc: 0.8920
Epoch 11/20
240/240 [================

In [3]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# specify the shear argument
shear_range_val = 45

# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# create the class object
datagenn = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val, shear_range=shear_range_val)

# fit the generator
datagenn.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagenn.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.19013890506625175 / Test accuracy: 0.9372


## Rotate + W&H Shift + Shear + Zoom

In [2]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# specify the shear argument
shear_range_val = 45

# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# specify the zoom argument
zoom_range_val=[2.5,3.5]

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val, shear_range=shear_range_val, zoom_range=zoom_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 12s 48ms/step - loss: 1.6562 - acc: 0.4207
Epoch 2/20
240/240 [==============================] - 9s 38ms/step - loss: 1.0986 - acc: 0.6337
Epoch 3/20
240/240 [==============================] - 9s 38ms/step - loss: 0.9077 - acc: 0.7016
Epoch 4/20
240/240 [==============================] - 9s 39ms/step - loss: 0.8080 - acc: 0.7357
Epoch 5/20
240/240 [==============================] - 10s 41ms/step - loss: 0.7306 - acc: 0.7599
Epoch 6/20
240/240 [==============================] - 10s 40ms/step - loss: 0.6773 - acc: 0.7763
Epoch 7/20
240/240 [==============================] - 10s 40ms/step - loss: 0.6358 - acc: 0.7920
Epoch 8/20
240/240 [==============================] - 10s 41ms/step - loss: 0.6040 - acc: 0.8011
Epoch 9/20
240/240 [==============================] - 10s 40ms/step - loss: 0.5692 - acc: 0.8162
Epoch 10/20
240/240 [==============================] - 10s 40ms/step - loss: 0.5494 - acc: 0.8202
Epoch 11/20
240/240 [===========

In [3]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# specify the shear argument
shear_range_val = 45

# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# specify the zoom argument
zoom_range_val=[2.5,3.5]

# create the class object
datagenn = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val, shear_range=shear_range_val, zoom_range=zoom_range_val)

# fit the generator
datagenn.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagenn.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.298855366563797 / Test accuracy: 0.905
